All India Management Association (AIMA) Extract Price

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# === Step 1: Define the fee extractor function ===
def get_all_aima_fees(url):
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    try:
        res = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(res.content, 'html.parser')

        fee_results = []

        # 1. Fees from <td><p> with Rs.
        td_tags = soup.find_all('td', {'valign': 'top', 'width': '312'})
        for td in td_tags:
            p_tag = td.find('p')
            if p_tag and 'Rs.' in p_tag.text:
                fee_results.append(p_tag.text.strip())

        # 2. <span> like "Online- Live Classes-" followed by <p> Rs.
        spans = soup.find_all('span')
        for span in spans:
            if 'Live Classes' in span.text or 'Face to Face' in span.text:
                next_p = span.find_next('p')
                if next_p and 'Rs.' in next_p.text:
                    label = span.text.strip()
                    price = next_p.text.strip()
                    fee_results.append(f"{label} {price}")

        # 3. <strong> with 'Program fee'
        strong_tags = soup.find_all('strong')
        for tag in strong_tags:
            if re.search(r'Program\s+fee', tag.text, re.I):
                fee_results.append(tag.text.strip())

        return ' | '.join(fee_results) if fee_results else '❌ Fee Not Found'

    except Exception as e:
        return f"❌ Error: {str(e)}"

# === Step 2: Load input Excel and process links ===
input_path = r"C:\Users\taslim.siddiqui\Downloads\All India Management Association (AIMA)_input.xlsx"
output_path = r"C:\Users\taslim.siddiqui\Downloads\All India Management Association (AIMA)_price.xlsx"

# Load course links from Excel
df = pd.read_excel(input_path)

# Add output column
df['Extracted Fee(s)'] = df['Course link'].apply(get_all_aima_fees)

# Save to output Excel
df.to_excel(output_path, index=False)

print("✅ Price extraction completed and saved to output Excel.")
